## Build the environment

In [3]:
#@test {"skip": true}

# tensorflow_federated_nightly also bring in tf_nightly, which
# can causes a duplicate tensorboard install, leading to errors.
!pip uninstall --yes tensorboard tb-nightly
!pip install --quiet --upgrade tensorflow-federated-nightly
!pip install --quiet --upgrade nest-asyncio
!pip install --quiet --upgrade tb-nightly  # or tensorboard, but not both

import nest_asyncio
nest_asyncio.apply()

Uninstalling tensorboard-2.4.1:
  Successfully uninstalled tensorboard-2.4.1
     |████████████████████████████████| 604kB 8.8MB/s 
     |████████████████████████████████| 194kB 19.2MB/s 
     |████████████████████████████████| 4.2MB 20.0MB/s 
     |████████████████████████████████| 51kB 6.7MB/s 
     |████████████████████████████████| 174kB 40.8MB/s 
     |████████████████████████████████| 448.5MB 41kB/s 
     |████████████████████████████████| 471kB 16.3MB/s 
     |████████████████████████████████| 1.3MB 30.2MB/s 
     |████████████████████████████████| 4.0MB 28.6MB/s 
     |████████████████████████████████| 5.9MB 32.3MB/s 
     |████████████████████████████████| 4.9MB 35.7MB/s 
ERROR: tensorflow 2.4.1 requires tensorboard~=2.4, which is not installed.
ERROR: tensorflow 2.4.1 has requirement gast==0.3.3, but you'll have gast 0.4.0 which is incompatible.
ERROR: tensorflow 2.4.1 has requirement grpcio~=1.32.0, but you'll have grpcio 1.37.1 which is incompatible.
ERROR: tensorflow 2.4.1

In [4]:
%load_ext tensorboard

In [5]:
import collections
import os
import numpy as np
import tensorflow as tf
import tensorflow_federated as tff
AUTOTUNE = tf.data.experimental.AUTOTUNE
import math, numpy as np
from keras.preprocessing import image
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras import optimizers


np.random.seed(0)

tff.federated_computation(lambda: 'Hello, World!')()

b'Hello, World!'

## Prepare the data

### Input the data

we use tiny imagenet dataset from GitHub

Here's how we can load it.

In [6]:
# download the dataset from GitHub
! git clone https://github.com/Torch-Dragon/41004-AI-Capstone-Project-.git

Cloning into '41004-AI-Capstone-Project-'...
remote: Enumerating objects: 90, done.
remote: Counting objects: 100% (90/90), done.
remote: Compressing objects: 100% (89/89), done.
remote: Total 90 (delta 37), reused 0 (delta 0), pack-reused 0
Unpacking objects: 100% (90/90), done.


In [7]:
! ls

41004-AI-Capstone-Project-  sample_data


In [8]:
#unzip dataset
! unzip -d ./Dataset ./41004-AI-Capstone-Project-/Dataset1.zip
! unzip -d ./Dataset ./41004-AI-Capstone-Project-/Dataset2.zip

流式输出内容被截断，只能显示最后 5000 行内容。
  inflating: ./Dataset/5/5_584.jpg   
  inflating: ./Dataset/5/5_779.jpg   
  inflating: ./Dataset/5/5_751.jpg   
  inflating: ./Dataset/5/5_989.jpg   
  inflating: ./Dataset/5/5_745.jpg   
  inflating: ./Dataset/5/5_976.jpg   
  inflating: ./Dataset/5/5_962.jpg   
  inflating: ./Dataset/5/5_792.jpg   
  inflating: ./Dataset/5/5_786.jpg   
  inflating: ./Dataset/5/5_619.jpg   
  inflating: ./Dataset/5/5_625.jpg   
  inflating: ./Dataset/5/5_143.jpg   
  inflating: ./Dataset/5/5_157.jpg   
  inflating: ./Dataset/5/5_631.jpg   
  inflating: ./Dataset/5/5_802.jpg   
  inflating: ./Dataset/5/5_816.jpg   
  inflating: ./Dataset/5/5_180.jpg   
  inflating: ./Dataset/5/5_194.jpg   
  inflating: ./Dataset/5/5_369.jpg   
  inflating: ./Dataset/5/5_341.jpg   
  inflating: ./Dataset/5/5_427.jpg   
  inflating: ./Dataset/5/5_433.jpg   
  inflating: ./Dataset/5/5_355.jpg   
  inflating: ./Dataset/5/5_382.jpg   
  inflating: ./Dataset/5/5_396.jpg   
  inflating: ./Dataset/

### Explore the content of the data set.

In [10]:
import random
from pathlib import Path
# store the images paths to lists
train_paths = []
valid_paths = []
test_paths = []

for i in range(10):
  image_paths = []
  path = Path("./Dataset/", str(i))
  for item in path.iterdir():
    image_paths.append(str(item))

  train_paths.extend(image_paths[:600])
  valid_paths.extend(image_paths[600:800])
  test_paths.extend(image_paths[800:1000])

In [11]:
print(train_paths[0])

Dataset/0/0_869.jpg


In [12]:
random.shuffle(train_paths)
random.shuffle(valid_paths)
random.shuffle(test_paths)

In [13]:
train_labels = []
valid_labels = []
test_labels = []

train_labels = [int(path[8:9]) for path in train_paths]
valid_labels = [int(path[8:9]) for path in valid_paths]
test_labels = [int(path[8:9]) for path in test_paths]

In [14]:
print(train_labels[:10])
print(valid_labels[:10])
print(test_labels[:10])

[1, 0, 4, 1, 1, 2, 0, 1, 7, 0]
[5, 4, 4, 3, 5, 7, 0, 5, 3, 0]
[1, 5, 6, 0, 5, 2, 5, 8, 1, 6]


### Preprocessing the data set

Build a function to preprocess the dataset,  
`preprocess_image `can resize and normalize the input data set.

In [30]:
#The preprocess function about resizing and normalizing images and paths
def preprocess_train_image(image):
  image = tf.image.decode_jpeg(image, channels=3)
  image = tf.image.resize(image, [64, 64]) #Resize all images
  #随机上下翻转
  image=tf.image.random_flip_left_right(image)
  #随机左右翻转
  image=tf.image.random_flip_up_down(image)
  #随机上下翻转图像
  image=tf.image.random_brightness(image,0.5)
  #随机改变对比度
  image=tf.image.random_contrast(image,0,1)
  
  #将图像进行归一化
  
  image /= 255.0  # normalize to [0,1] range

  return image

def preprocess_image(image):
  image = tf.image.decode_jpeg(image, channels=3)
  image = tf.image.resize(image, [64, 64]) #Resize all images
  
  image /= 255.0  # normalize to [0,1] range
  return image

def load_and_preprocess_train_image(path):
  image = tf.io.read_file(path)
  return preprocess_train_image(image)

def load_and_preprocess_image(path):
  image = tf.io.read_file(path)
  return preprocess_image(image)

In [31]:
# load and preprocess the images
train_images = [load_and_preprocess_train_image(path) for path in train_paths]
                                          
valid_images = [load_and_preprocess_image(path) for path in valid_paths]

test_images = [load_and_preprocess_image(path) for path in test_paths]


# tranfer to numpy.array

train_images = np.asarray(train_images)
train_labels = np.asarray(train_labels)

valid_images = np.asarray(valid_images)
valid_labels = np.asarray(valid_labels)

test_images = np.asarray(test_images)
test_labels = np.asarray(test_labels)



In [32]:
print("train_images[0]:")
print(train_images[2])
print("train_labels[0]:")
print(train_labels[0])

train_images[0]:
[[[0.8823852  0.8907406  0.50297016]
  [0.90514284 0.90049386 0.51272345]
  [0.9148962  0.8972428  0.4932168 ]
  ...
  [0.9148962  0.8907406  0.50297016]
  [0.90839404 0.90049386 0.48021245]
  [0.9116451  0.90374494 0.47696134]]

 [[0.8986407  0.8972428  0.5192256 ]
  [0.8888874  0.8744851  0.4932168 ]
  [0.8531253  0.8159653  0.42494377]
  ...
  [0.85962754 0.8094631  0.44770145]
  [0.86612976 0.84197414 0.46070588]
  [0.9116451  0.8972428  0.50622123]]

 [[0.9148962  0.8907406  0.5289789 ]
  [0.742588   0.70217687 0.33716413]
  [0.9148962  0.84197414 0.47045916]
  ...
  [0.8531253  0.7411901  0.44119927]
  [0.6060419  0.5493753  0.22987789]
  [0.9148962  0.8874895  0.5582388 ]]

 ...

 [[0.90839404 0.8874895  0.50622123]
  [0.7523413  0.71843237 0.3404152 ]
  [0.775099   0.705428   0.3501685 ]
  ...
  [0.775099   0.806212   0.50297016]
  [0.5670287  0.56888187 0.22987789]
  [0.8986407  0.8874895  0.5289789 ]]

 [[0.9148962  0.90049386 0.48346353]
  [0.9116451  0.8809

In [33]:
print(train_images.shape)

(6000, 64, 64, 3)


#Federated Learning

Question: We have used FedAvg here. But what do we need to do to improve the model performance if we implement Non-IID setting ?

In [34]:
BATCH_SIZE = 32
NUM_CLIENTS = 20
SAMPLE_CLIENTS = 10

def create_tf_dataset_for_client(client_id):
  client = tf.data.Dataset.from_tensor_slices((train_images[client_id*300: (client_id+1)*300], train_labels[client_id*300: (client_id+1)*300]))
  client = client.batch(BATCH_SIZE)
  client = client.prefetch(buffer_size=AUTOTUNE)
  return client

In [35]:
def federated_train_data():
  client_ids = np.random.choice(range(0,NUM_CLIENTS), size=SAMPLE_CLIENTS, replace=False)
  data= [create_tf_dataset_for_client(x)
  for x in client_ids
  ]
  return data

In [36]:
def federated_valid_data():
  valid_set = tf.data.Dataset.from_tensor_slices((valid_images, valid_labels))
  valid_set = valid_set.batch(BATCH_SIZE)
  valid_set = valid_set.prefetch(buffer_size=AUTOTUNE)
  data = [valid_set]
  return data

In [37]:
def federated_test_data():
  test_set = tf.data.Dataset.from_tensor_slices((test_images, test_labels))
  test_set = test_set.batch(BATCH_SIZE)
  test_set = test_set.prefetch(buffer_size=AUTOTUNE)
  data = [test_set]
  return data

In [38]:
input_spec = federated_train_data()[0].element_spec

In [73]:
def create_keras_model():
  model = tf.keras.Sequential()
  model.add(tf.keras.layers.Conv2D(32, kernel_size=(3,3), activation='relu', input_shape=(64, 64, 3)))
  model.add(tf.keras.layers.MaxPool2D(pool_size=(2,2), strides=(2,2)))
  model.add(tf.keras.layers.Dropout(0.5))
  model.add(tf.keras.layers.Conv2D(64, kernel_size=(3,3), activation='relu'))
  model.add(tf.keras.layers.MaxPool2D(pool_size=(2,2), strides=(2,2)))
  model.add(tf.keras.layers.Dropout(0.5))
  model.add(tf.keras.layers.Conv2D(64, kernel_size=(3,3), activation='relu'))
  model.add(tf.keras.layers.MaxPool2D(pool_size=(2,2), strides=(2,2)))
  model.add(tf.keras.layers.Dropout(0.5))
  model.add(tf.keras.layers.Flatten())
  model.add(tf.keras.layers.Dense(1000, activation='relu'))
  model.add(tf.keras.layers.Dense(10, activation='softmax'))
  
  return model


Question: We have used SparseCategoricalCrossentropy which you did not suggest. And I have tried MSE and MAE. But they did not behave well. Would like to ask What loss function do you advise.

In [52]:
def model_fn():
  # We _must_ create a new model here, and _not_ capture it from an external
  # scope. TFF will call this within different graph contexts.
  keras_model = create_keras_model()
  return tff.learning.from_keras_model(
      keras_model,
      input_spec=input_spec,
      loss=tf.keras.losses.SparseCategoricalCrossentropy(),
      metrics=[tf.keras.metrics.SparseCategoricalAccuracy()])

In [74]:
iterative_process = tff.learning.build_federated_averaging_process(
    model_fn,
    client_optimizer_fn=lambda: tf.keras.optimizers.SGD(learning_rate=0.01),
    server_optimizer_fn=lambda: tf.keras.optimizers.SGD(learning_rate=1.0))

In [75]:
str(iterative_process.initialize.type_signature)

'( -> <model=<trainable=<float32[3,3,3,32],float32[32],float32[3,3,32,64],float32[64],float32[3,3,64,64],float32[64],float32[2304,1000],float32[1000],float32[1000,10],float32[10]>,non_trainable=<>>,optimizer_state=<int64>,delta_aggregate_state=<value_sum_process=<>,weight_sum_process=<>>,model_broadcast_state=<>>@SERVER)'

In [77]:
#@test {"skip": true}
logdir = "/tmp/logs/scalars/training/"
summary_writer = tf.summary.create_file_writer(logdir)
state = iterative_process.initialize()

In [78]:
evaluation = tff.learning.build_federated_evaluation(model_fn)

In [ ]:
import time

NUM_ROUNDS = 200
#@test {"skip": true}
with summary_writer.as_default():
  for round_num in range(NUM_ROUNDS):
    start = time.time()
    state, metrics = iterative_process.next(state, federated_train_data())
    end = time.time()
    valid_metrics = evaluation(state.model, federated_valid_data())

    train_metrics = {}
    train_metrics['accuracy'] = metrics['train']['sparse_categorical_accuracy']
    train_metrics['loss'] = metrics['train']['loss']
    train_metrics['val_accuracy'] = valid_metrics['sparse_categorical_accuracy']
    train_metrics['val_loss'] = valid_metrics['sparse_categorical_accuracy']
    
    for name, value in train_metrics.items():
      tf.summary.scalar(name, value, step=round_num+1)

    print('round', round_num+1, round(end-start), 's, ', 
          'accuracy=', metrics['train']['sparse_categorical_accuracy'],
          'loss=', metrics['train']['loss'],
          'val_accuracy=', valid_metrics['sparse_categorical_accuracy'],
          'val_loss=', valid_metrics['loss'])
                                                                              

In [ ]:
#@test {"skip": true}
!ls {logdir}
%tensorboard --logdir {logdir} --port=0

In [ ]:
#@test {"skip": true}
!rm -R /tmp/logs/scalars/*

In [ ]:
test_metrics = evaluation(state.model, federated_test_data())
print(test_metrics)

OrderedDict([('sparse_categorical_accuracy', 0.593), ('loss', 1.1702322)])
